# Rag - система для проекта Physical Transformation
### В  данном ноутбуке представлена RAG-Система для проекта Physical Transformation, которая поможет получать быстрые ответы на вопросы, опираясь на базу знаний проекта. База знаний проекта представляет из себя только статьи, находящиеся в открытом доступе. Статьи были собраны и обработаны вручную. 
### На данный момент, нет универсального решения, которое поможет оценить производительность RAG системы. В данном проекте мы попробуем оценить систему используя общепринятые и кастомные метрики. 
P.S. Библиотека RAGAS с русским языком требует тонкой настройки, что в kaggle или colab сделать не получается. [ссылка](https://habr.com/ru/companies/sberbank/articles/831346/)

# Импорт необходимых библиотек


In [ ]:
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers openpyxl pacmap datasets langchain-community ragatouille pacmap langchain-huggingface
!pip install -q gdown torch transformers transformers accelerate bitsandbytes langchain sentence-transformers openpyxl datasets langchain-community ragatouille umap-learn
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -q gdown faiss-gpu umap-learn

In [ ]:
import gdown
import pandas as pd
from tqdm.notebook import tqdm
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
from langchain.docstore.document import Document as LangchainDocument
#from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import matplotlib.pyplot as plt
import pacmap
import numpy as np
import plotly.express as px
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from ragatouille import RAGPretrainedModel
import warnings
from datasets import Dataset


warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.set_option("display.max_colwidth", None)  # Полезно при визуализации результатов поиска

# Работа с данными, эмбеддинги, создание базы знаний.
### Данные из себя представляют статьи на русском. Статьи имеют практически одинаковую структуру. Все статьи были собраны вручную. 

In [ ]:
import os
dataset_path = '/kaggle/input/state-physical'
s = 'Статья '
texts = []


for i in range (1, 59):
  file_path = os.path.join(dataset_path, f'{i}.txt') 
  my_dict = dict({})
  with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
    my_dict['название статьи'] = (text.split('\n'))[0]
    flag = False
    sources = []
    flag2 = 0
    for i, line in enumerate(text.split('\n')[-100:]):
      if 'Автор' in line:
        flag2 = i
        my_dict['автор статьи'] = line
      else:
        my_dict['автор статьи'] = '-'
      if flag == True:
        sources.append(line)
      if 'Список источников' in line:
        flag = True

    my_dict['Список источников'] = ('').join(sources)
    vsp = (text.split('\n'))[2:-100+flag2]
    my_dict['текст'] = [k.strip() for k in vsp if k!='']
  texts.append(my_dict)


In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [LangchainDocument(page_content=('\n').join(state["текст"]), metadata={"название статьи": state["название статьи"], "автор статьи": state["автор статьи"], "Список источников": state["Список источников"]}) for state in texts]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Мы используем иерархический список разделителей, специально предназначенных для разделения документов Markdown
# Этот список взят из класса MarkdownTextSplitter в LangChain

MARKDOWN_SEPARATORS = [
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # Максимальное количество символов в чанке
    chunk_overlap=100,  # Количество символов, которые будут перекрываться между чанками
    add_start_index=False,  # Если "True", то включает начальный индекс чанка в метаданные
    strip_whitespace=True,  # Если значение "True", то пробелы удаляются из начала и конца каждого документа
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
k=0
for doc in RAW_KNOWLEDGE_BASE:
    for docum in text_splitter.split_documents([doc]):
        k+=1
        new_metadata = docum.metadata.copy()
        new_metadata["номер_чанка"] = k
        updated_chunk = LangchainDocument(
            page_content=docum.page_content,
            metadata=new_metadata
        )
        docs_processed.append(updated_chunk)




In [ ]:
from sentence_transformers import SentenceTransformer

# Чтобы получить значение max sequence_length, мы запросим базовый объект `SentenceTransformer`, используемый в RecursiveCharacterTextSplitter
print(f"Model's maximum sequence length: {SentenceTransformer('cointegrated/LaBSE-en-ru').max_seq_length}")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

# Построим график распределения длин документов, подсчитываемых как количество токенов
fig = pd.Series(lengths).hist()
plt.title("Распределение длин документов в базе знаний (в количестве токенов)")
plt.show()

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True} )

In [ ]:
from langchain_chroma import Chroma

vectors_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding
)

In [ ]:
from langchain_core.documents import Document
embeddings_for_bd = embedding.embed_documents( [i.page_content for i in (docs_processed)])
ids_r = [str(i) for i in range(len(docs_processed))]
vectors_store.add_documents(
    documents=docs_processed
)

In [ ]:
user_query = "Сколько нужно пить жидкости??"
query_vector = embedding.embed_query(user_query)
embs = vectors_store.get(include=['embeddings'])['embeddings']

embedding_projector = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, random_state=1)

documents_projected = embedding_projector.fit_transform(np.array(list(embs)+[query_vector]), init="pca")

In [ ]:
df = pd.DataFrame.from_dict(
    [
        {
            "x": documents_projected[i, 0],
            "y": documents_projected[i, 1],
            "source": 'db',
            "extract": docs_processed[i].page_content[:100] + "...",
            "symbol": "circle",
            "size_col": 4,
        }
        for i in range(len(docs_processed))
    ]
    + [
        {
            "x": documents_projected[-1, 0],
            "y": documents_projected[-1, 1],
            "source": 'user_query',
            "extract": user_query,
            "size_col": 50,
            "symbol": "star",
        }
    ]
)

fig = px.scatter(
    df,
    x="x",
    y="y",
    color="source",
    hover_data="extract",
    size="size_col",
    symbol="symbol",
    color_discrete_map={"User query": "black"},
    width=1000,
    height=700,
)
fig.update_traces(
    marker=dict(opacity=1, line=dict(width=0, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(
    legend_title_text="<b>Источник чанка</b>",
    title="<b>2D-проекция вложений чанка с помощью PaCMAP</b>",
)
fig.show()

# Оценка качества Ретривера 

### Теперь у нас есть ретривер и по запросу мы можем найти k-ближайших статей. Извлечение релевантных документов одна из важнейших частей RAG системы. И мы хотим оценить её качество. Для этого сделаем следующее:
- Во-первых, нам нужны вопросы, ответы на которые должна будет дать сисетма. Для оценки нам нужно много вопросов, писать вручную их долго и сложно. Попросим LLM сделать это.
- Во-вторых: вопросы будут формироваться на основе чанков текстов. Очевидно, часть вопросов будут некорректными. Потому попросим другую LLM отфильтровать вопросы. [вдохновлялся вот здесь](https://huggingface.co/learn/cookbook/en/rag_evaluation)
- В-треьих: Используя вопросы, оценим качество Ретривера (релевантность извлеченных документов) с помощью классических и кастомных метрик. 

# Генерация пар вопрос - ответ для оценки модели 

### 1)

In [ ]:
#Я выбрал модель: T-lite-it-1.0. Нашел её в лидербрде, она показала хорошие результаты на датасете QA. Я подумал, что из нетяжелых моделей, она может показать приемлемое качество. 
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

device = 'cuda'
model_name = "t-tech/T-lite-it-1.0"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Используем 4-битное квантование
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    torch_dtype="auto",
    device_map=device
)

model = model.to(device)
messages = [
    {
        "role": "system",
        "content": """
Ваша задача — написать вопрос-фактоид и ответ на него с учетом контекста.
Ваш вопрос-фактоид должен содержать конкретную, краткую фактическую информацию из контекста.
Ваш вопрос-фактоид должен быть сформулирован в том же стиле, что и вопросы, которые могут задавать люди, интересующиеся фитнесом и правильным питанием.
Это означает, что ваш вопрос-фактоид НЕ ДОЛЖЕН упоминать что-то вроде «согласно отрывку» или «контекст».

Предоставьте свой ответ следующим образом:

Output:::
Вопрос-фактоид: (ваш вопрос-фактоид)
Ответ: (ваш ответ на вопрос-фактоид)
Вот контекст. Context:
{context}\n
"""}
]

In [ ]:
Q_A = []
for i in tqdm(docs_processed):
  model = model.to(device)
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )
  final_prompt = text.format(context=i.page_content)
  model_inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
  generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256
  )
  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]
  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  question = response.split("Фактоидный вопрос: ")[-1].split( "Ответ: ")[0]
  answer = response.split( "Ответ: " )[- 1 ]
  print(response)
  Q_A.append({'context': i.page_content, 'question':question, 'answer': answer, 'state': i.metadata['название статьи']})


In [ ]:
#Сохраним файл с вопросами
quest_answer = pd.DataFrame.from_dict(Q_A)
excel_file_path = '/kaggle/working/my_dataframe.xlsx'
quest_answer.to_excel(excel_file_path, index=False)

In [ ]:
# Скачаем файл с вопросами. 
dataset_path = '/kaggle/input/questandanswer/my_dataframe.xlsx' #Если что, нужно заменить на свою директорию 
Q_A = pd.read_excel(dataset_path)

In [ ]:
Q_A.head()

### 2)
Мы получили 147 вопросов. Попробуем попросить другую языковую модель оценить их. Для этого введем 3 критерия: 

In [ ]:
# Оценка, можно ли ответить на вопрос, исходя из данного контекста?
question_groundedness_critique_prompt = """
Вам будет предоставлен контекст и вопрос.
Ваша задача — предоставить «общую оценку», показывающую, насколько хорошо можно однозначно ответить на заданный вопрос с заданным контекстом.
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что на вопрос вообще невозможно ответить с учетом контекста, а 5 означает, что на вопрос можно четко и однозначно ответить с учетом контекста.

Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 1 до 5
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Контекст прямо содержит информацию о причинах события.
Оценка: 5

Ниже представлен вопрос и контекст.

Question: {question}
Context: {context}

Output:::
"""
#Оценка, насколько вопрос может быть актуален для пользователя
question_relevance_critique_prompt = """
Вам будет задан вопрос.
Ваша задача — предоставить «общую оценку», отражающую, насколько полезен этот вопрос для людей, которые хотят разобраться в сфере правильного питания, фитнеса и здорового образа жизни.
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что вопрос вообще бесполезен, а 5 означает, что вопрос чрезвычайно полезен.

Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 1 до 5
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Вопрос актуален для человека, который интересуется здоровьем
Оценка: 5

Ниже представлен вопрос.

Question: {question}

Output:::
"""
#Оценка, не используется ли в вопросе контекст
question_standalone_critique_prompt = """
Вам будет задан вопрос.
Ваша задача — предоставить «общую оценку», представляющую, насколько этот вопрос независим от контекста.
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что вопрос зависит от дополнительной информации для понимания, а 5 означает, что вопрос сам по себе имеет смысл.

Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 1 до 5
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Вопрос независим от контекста
Оценка: 5

Ниже представлен вопрос.

Question: {question}

Output:::
"""

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(Q_A):
    #print(output["question"])
    #print(output["context"])
    prompt_groundedness = question_groundedness_critique_prompt.format(question=output["question"], context=output["context"]),
    model_inputs_1 = tokenizer(prompt_groundedness, return_tensors="pt").to(model.device)
    generated_ids_groun = model.generate(
      **model_inputs_1,
      max_new_tokens=80
    )
    generated_ids_groun = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs_1.input_ids, generated_ids_groun)
    ]
    groundedness = tokenizer.batch_decode(generated_ids_groun, skip_special_tokens=True)[0]
    print(groundedness)
    #print('---------------')


    prompt_relevance = question_relevance_critique_prompt.format(question=output["question"])
    model_inputs_2 = tokenizer(prompt_relevance, return_tensors="pt").to(model.device)
    generated_ids_rel = model.generate(
      **model_inputs_2,
      max_new_tokens=80
    )
    generated_ids_rel = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs_2.input_ids, generated_ids_rel)
    ]
    relevance = tokenizer.batch_decode(generated_ids_rel, skip_special_tokens=True)[0]
    print(relevance)
    #print('---------------')


    prompt_standalone = question_standalone_critique_prompt.format(question=output["question"])
    model_inputs_3 = tokenizer(prompt_standalone, return_tensors="pt").to(model.device)
    generated_ids_st = model.generate(
      **model_inputs_3,
      max_new_tokens=80
    )
    generated_ids_st = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs_3.input_ids, generated_ids_st)
    ]
    standalone = tokenizer.batch_decode(generated_ids_st, skip_special_tokens=True)[0]
    print(standalone)
    #print('---------------')


    evaluations = {"groundedness":groundedness, "relevance": relevance, "standalone": standalone}

    try:
      for criterion, evaluation in evaluations.items():

            score, eval = (
                int(evaluation.split("Оценка: ")[-1].strip()[0]),
                evaluation.split("Оценка: ")[-2].split('Комментарий:')[1],
            )
            print (score)
            print (eval)
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
      print('+1')
    except:
        print('-1')
        continue

### Если посмотреть выводы выше, можно увидеть, что модель часто галлюцинирует. Я пробовал около 10 промптов, выше оставил самые лучшие. Далее проверим работу модели.

In [ ]:
#Я добавил ячейку, чтобы подгружать данные при повторном воспроизведении ноутбука 
path = '/kaggle/input/questandanswer/evalQA.xlsx'
Q_A_eval = pd.read_excel(path)

In [ ]:
Q_A_eval.head(6)

In [ ]:
#Отфильтруем датасет. Оставим только вопросы, с оценками >=4
tr = Q_A_eval[(Q_A_eval['relevance_score'] >= 4) & (Q_A_eval['standalone_score'] >= 4) & (Q_A_eval['groundedness_score'] >= 4)]
tr[['question', 'relevance_score', 'standalone_score', 'groundedness_score']].head(20)

### Можем видеть, что некоторые вопросы некорректные (например 13, 36). И так как вопросов не очень большое количество, я отфильтрую их вручную. (При этом, система с 3-мя оценками отработала очень не плохо.

### Очень важное примечание. Мы не оцениваем эталонные ответы на данном этапе. Так как нас интересует только ретривер, который не зависит от корректности ответов. 

In [ ]:
indexes = [3, 5, 6, 10, 14, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 41, 42, 44, 45, 46, 47, 49, 51, 52, 53, 55, 56, 59, 62, 64, 66, 67, 68, 70, 71, 73, 75, 77, 80, 89, 92, 95, 97, 98, 99, 100, 101, 106, 108, 109, 110, 113, 115, 117, 119, 120, 121, 123, 126, 128, 129, 130, 131, 132, 133, 135, 136, 137, 140, 141, 142, 143, 143, 145, 146]
questions = Q_A_eval.loc[indexes]['question']

In [ ]:
questions

# Метрики, для оценки Ретривера 

### Когда мы имеем список вопросов. Мы можем оценить качество нашего ретривера. 
Здесь все сильно не однозначно. Во-первых, мы будем использовать выбранную метрику, чтобы выбрать лучший пайплайн, оптимизировать гиперпараметры и т.д. Классические context precision recall сильно зависят от k-количества выбранных чанков ретривером, также будут зависеть от длины чанка. Модель с большим количеством чанков всегда будет уступать по context recall, так как в нашей небольшой базе знаний может не быть k релевантных документов. 
ROUGE, BLEU тоже не лучший выбор, так как тоже зависят от длины чанка, а также требуют эталонного ответа. А наши эталонные ответы были сформулированы по одному чанку, при этом в других чанках может быть тоже релевантная информация, которая не имеет ничего общего с информацией в эталонном ответе. 

Пример: 
Вопрос: Какие группы продуктов способствуют насыщению? 
Чанк 1: Белковые продукты имеют низкую энергетическую плотность, поэтому после них долго не хочется есть. 
Чанк 2: Картофель имеет самый высокий индекс насыщения согласно исследованию [11] 
Чанк 3: Овощи содержат большое количество клетчатки, потому долго остаются в ЖКТ и, как следствие насыщают. 
Итого: Один из чанков получит высокое значение метрики, а остальные чанки будут считаться нецелевыми, так как будут "далеко" от эталонного. 

Итак, для оценки будем использовать 3 метрики, чтобы получить максимально объективную оценку:

Для любого вопроса можем получить id чанка, по которому он был сформулирован. Давай оценивать, найдет ли ретривер тот чанк, из которого был сформулирован вопрос. И требовать, чтобы этот чанк был как можно выше в ранжированном списке. 
Итого, первая метрика: DCG@K $$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
Далее, можно посмотреть семантическое сходство чанков и вопроса. Использовать эмбеддинги для этого выглядит нелогично, так как мы и так получим наиболее близкие по косинусному расстоянию чанки, поэтому оценим сходство с помощью отдельной модели. 
И третье: попросим LLM оценить, насколько каждый из выданных чанков помогает ответить на вопрос. 


Я попробовал оценить context precision и context recall. Столкнулся со следующими проблемами: Библиотека RAGAS очень плохо работает с русским языком и требует большого количества сложных настроек. А самое главное, не получается её запустить на Каггл или Колаб с нормальными LLM. 
Далее я попробовал реализовать их вручную с помощью промпта (Дам вопрос и документ, скажи, помогает ли документ ответить на вопрос). Модели 7-8 млрд параметров хорошо работают только тогда, когда передаешь 1 чанк и просишь оценить только его релевантность(если чанков 5, то на 1 сэмпл уходит 10 мин времени). Модели 1-2 млрд параметров работают никак. Поэтому метрики нужно было разрабатывать вручную. P.S. Это еще один аргумент, почему я не взял классические метрики. 

In [ ]:
# оставил свои попытки сделать recall precision ниже, вдруг вернусь еще к этому. 

In [ ]:
questions = Q_A_eval.loc[indexes]
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загрузка модели и токенизатора
model_name = "Vikhrmodels/Vikhr-Llama-3.2-1B-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)
prompt = """
Вам будет предоставлен вопрос и документ. 
Ваша задача - оценить, одержит ли документ информацию, которая поможет ответить на вопрос.  Дайте свой ответ в формате 0 или 1. Где 1 означает, что документ полезен, а 0 означает, что документ бесполезен. 


Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 0 до 1
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Документ бесполезен для ответа на вопрос
Оценка: 0

Ниже представлен вопрос и документ

Вопрос: {question}
Документ: {context}

Output:::
"""

for (i,serie) in  questions.iterrows():
    print(i)
    if i>10:
        break
    question = serie['question']
    user_query = question
    retrieved_docs = vectors_store.similarity_search(query=user_query, k=5)
    for doc in retrieved_docs:
        context = doc.page_content
        inp = prompt.format(question=question, context=context)
        input_ids = tokenizer.encode(inp, return_tensors="pt")
        input_ids = input_ids.to(device)
        output = model.generate(
          input_ids,
          max_length=1000,
          temperature=0.95,
          num_return_sequences=1,
          no_repeat_ngram_size=2,
          top_k=50,
          top_p=0.95,
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print (generated_text)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import BitsAndBytesConfig

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


context_critique_prompt = """
Вам будет предоставлен вопрос и контекст. Определите, дает ли контекст достаточно информации для ответа на вопрос.

**Инструкции:**
1. Ответ должен содержать ТОЛЬКО блоки "Комментарий" и "Оценка".
2. Не добавляйте посторонний текст до или после этих блоков.
3. Оценка — число от 0 до 1 с 1 знаком после запятой (например, 0.7).
4. Пример правильного ответа:
---
Комментарий: Контекст объясняет метод, но не дает конкретных данных.
Оценка: 0.5
---

Вопрос: {question}
Контекст: {context}

**Ваш ответ (только два блока):**
Комментарий: 
Оценка: 
"""

MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"
device = 'cuda'

model_critiqe = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model_critiqe.eval()

tokenizer_critiqe = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Critique = pipeline(
    model=model_critiqe,
    tokenizer=tokenizer_critiqe,
    task="text-generation",
    do_sample=True,
    temperature=1,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
)


In [ ]:
import re
results = []
for text, i in zip(questions, questions.index):
    retrieved_docs = vectors_store.similarity_search(query=text, k=5)
    #К индексу из questions прибавляется единица, так как нумерация отличается на 1, относительно базы знаний
    correct = [(i+1) == (doc.metadata['номер_чанка']) for doc in retrieved_docs] #Проверяем, каким по счету попал наш чанк в ранжированный список, если вообще попал
    try:
        position = (correct.index(1))+1 
        dcg_score = (position<=k) * 1/(np.log2(1+position))
    except: 
        dcg_score =0

    similarity_score = 0
    for o, doc in enumerate(retrieved_docs):
        weights = [0.3, 0.25, 0.2, 0.15, 0.1]
        text1 = doc.page_content
        out = model(**tokenizer(text1, text1, return_tensors='pt', truncation=True).to(model.device))

        proba = torch.softmax (out.logits.cpu().detach(), -1)[0] #Выводит 3 числа (в-ти 3х классов), где первый класс = второй текст следствие первого
        entailment = proba[0] 
        similarity_score += weights[o]*entailment

    
    context = "Извлеченный текстовый блок:"
    context += "".join([f"Document {str(j)}:::\n" + doc.page_content for j, doc in enumerate(retrieved_docs)])
    prompt = context_critique_prompt.format(question=text, context=context)
    answer = Critique(prompt)[0]["generated_text"]
    parts = answer.split("Оценка:")
    if len(parts) >= 2:
        answer = f"Оценка: {parts[-1].strip()}"
    match = re.search(r"Оценка:\s*([0-1]\.?\d*)", answer)
    try: 
        llm_score = float(match.group(1))
    except: 
        llm_score = None 

    new_row = {
        'id': i,
        'dcg_score': dcg_score,
        'entailment': similarity_score,
        'llm_score': llm_score
    }
    results.append(new_row)
    print(new_row)

df_evaluate = pd.DataFrame(results)
df_evaluate['entailment'] = df_evaluate['entailment'].apply(lambda x: x.item() if isinstance(x, torch.Tensor) else x)

### Интерпретация результатов оценки

In [ ]:
#Итого, мы получили оценку нашего ретривера. Во первых, хочется посмотреть, насколько скоррелированы значения метрик. 
corr = (df_evaluate[['dcg_score', 'entailment', 'llm_score']].corr())
corr.style.background_gradient(cmap='coolwarm')
#Видим полнейшее отсутствие корреляции 'entailment' с остальными метриками. Чтобы оценить, репрезентативна ли эта метрика сделаем следующеею.
#Ниже в ячейках я нашел несколько сэмплов, в которых очень большая разброс между метриками и вручную оценил эти сэмплы. Ниже пример.

In [ ]:
index_difs = []
for i, line in df_evaluate.iterrows():
    if line['llm_score']:
        dif = float(line['dcg_score']) + float(line['llm_score']) - float(line['entailment'])
        d = {line['id']: dif}
        index_difs.append(d)
index_difs

In [ ]:
df_evaluate[df_evaluate['id']==136]

In [ ]:
idxs = [136, 38, 132]
for i in idxs:
    print(questions[i])
    a = vectors_store.similarity_search(query=questions[i], k=5)
    for i in (a):
        print(i.page_content)
        print('-----')
    break

### Приведу пример 136 элемента нашего списка. На нем: dcg_score = 1 entailment = 0.669792 llm_score = 0.8. 


Вопрос-фактоид: Как влияет включение «вредных и бесполезных» быстрых углеводов в рацион на долгосрочную потерю веса?

1 Сэмпл:
А теперь пройдёмся по фактам: во-первых, углеводы безопасны как класс.
Низкоуглеводные диеты несут ноль преимуществ как для жиросжигания, так и для композиции [2]. Во-вторых, с точки зрения здоровья и долголетия высокоуглеводные диеты (50-60% на углеводы) несколько выигрывают [3]. В плане влияния на микрофлору — тоже (даже с высоким содержанием «условно простых») [4].
Гликемический и инсулиновый индексы углеводов (цельное / обработанное) для результата тоже не имеют значения. Они важны для диабетиков. Есть исследования на эту тему [5], [6], [7].
При уравнённом дефиците калорий и сходных БЖУ группы с высоким и низким содержанием сахара в рационе (потребители цельных и обработанных углеводов) теряют одинаковое количество веса и жира [8], [9].
В защиту того, что лучше включить сколько-то «вредных и бесполезных» быстрых углеводов, чем не включить, выступает исследование [10]. В нём сравнивают типичный ЗОЖ-ПП (низкие и очень «здоровые» углеводы) и диету с вписыванием вкусного и неЗОЖного (шоколад, пироженка, пончик и т. д.— уже не такие низкие и совсем не ЗОЖные угли!). Потеря веса была прогнозируемо сопоставимой, а самое интересное началось дальше!
Группа ЗОЖ-ПП после диеты вернулась к привычному питанию и знатно откатила. Группа, которая вписывала вкусное и неЗОЖное, не только продержалась, но и улучшила результаты после диеты. Потому что научилась вписывать вкусное в повседневный рацион. Вкус и удовольствие от пищи играют порой критически важную роль!


2 Сэмпл:
Протеин
Я отношу протеиновый порошок к этому списку оттого, что его воспринимают как добавку, спрашивают о рисках для печени и потенции, интересуются, можно ли его дважды в день, а не единожды.  Складывается впечатление, что протеин — это не добавка, а даже препарат, но нет. Прот — это просто еда.
Что делает: помогает добрать норму белка в дополнение ко всей остальной пище.
Удобный формат для тех, кто не может набрать норму в силу целого ряда причин: не хватает времени на готовку и организацию приёмов пищи, надо быстренько перекусить на ходу или просто аппетит не позволяет есть больше.
Здесь нет никаких эффектов быстрого похудения или увеличения силы и производительности. Очень подробно про белок (кому, зачем, сколько) я рассказал в своем ролике на Ютуб «Раскрываем популярный миф про белок».
Как же рассчитать, сколько белка нужно тебе?
Ориентируемся на эти нормы:
Сколько добирать протеином?
Простой пример. Допустим, с учётом активности и веса тебе нужно ежедневно наедать 135 г белка. В какой-то день из пищи ты набрал(а) 110 г. Не хватает 25 г — это и будем добирать протеином.
Старайся не набирать протеином больше 1-2 порций (25-50 г) белка в день. Вреда не будет, но больше — скорее признак несбалансированного рациона.
Омега-3
Здесь имеется в виду омега-3 в капсулах.
Что делает: помогает поддерживать сердечно-сосудистое здоровье, снижает триглицериды и уровень хронического воспаления [8-10].


3 Сэмпл: 
Почему «докинуть вкусного» не просто можно, но даже нужно? И вообще научиться диетить вкусно и разнообразно? В исследовании [2] одна группа диетила «по всей строгости», исключая сладкое и мучное, другая добавляла к приёму пищи десерты. По итогу 16 недель группы похудели одинаково успешно (строгая даже чуть успешнее), интересное случилось после. Первые за последующие четыре месяца откатили к исходным позициям, а вторые не только удержали результат, но даже улучшили его (ещё почти на семь кило!).
Что за магия десертов, спросите вы? А нет никакой магии. Первые отмучились с диетой и вернулись к привычному образу жизни/питания, с ним же вернулся и вес. Вторые научились диетить сбалансированно и с удовольствием, поэтому просто продолжили в том же духе.
Мы можем безболезненно позволить себе вкусное, потому что диета — гибкая. Именно на ней базируется Физикл. Она подразумевает системность, подсчёт (в большей или меньшей степени), не делит продукты на белые и чёрные и не запрещает какие-либо продукты как класс. Гибкая диета стала популярной в 2005 году благодаря книге Лайла Макдональда «А guide to flexible dieting» и совершенно расцвела вместе с распространённостью гаджетов.
...

### Тут Ретривер неплохо справился с задачей. Хотя, 2й сэмпл лишний. Но на этом фрагменте метрика 'entailment' одна из самых худших среди всех примеров. Отсюда делаем вывод, что 'entailment' плоховато отражает действительность. Мы не перестанем ее учитывать. Просто основные выводы будем делать по dcg и llm score

In [ ]:
print(df_evaluate['dcg_score'].mean())
print(df_evaluate['entailment'].mean())
print(df_evaluate['llm_score'].mean())

# Итоговая оценка Ретривера и лучшие гиперпараметры

### Таким образом, мы получили некоторые метрики, которые помогут АВТОМАТИЧЕСКИ оценивать качество извлечения. И теперь можем проварьировать параметры, чтобы подобрать оптимальные. Здесь я представлю таблицу гиперпараметров и метрик

| Модификация | dcg_score | llm_score | entailment
|-------------|-------------|-------------|-----------
| Размер чанка = 200 | | |
|Ранжирование = True | 0.14    | 0.48    |  0.62
| Модель эмбеддингов = cointegrated/rubert-tiny2  | | | 
|  | | | 
| Размер чанка = 200 | | |
|Ранжирование = True | 0.18    | 0.54   |  0.59
| Модель эмбеддингов = cointegrated/LaBSE-en-ru | | | 
|  | | | 
| Размер чанка = 500 | | |
|Ранжирование = True| 0.32    | 0.54    |  0.75
| Модель эмбеддингов = cointegrated/LaBSE-en-ru  | | | 
|  | | | 
| Размер чанка = 500 | | |
|Ранжирование = False| 0.44    | 0.58    |  0.79
| Модель эмбеддингов = cointegrated/LaBSE-en-ru| | | 
|  | | | 
| Размер чанка = 1500 | | |
|Ранжирование = False| 0.50    | 0.66    |  0.78
| Модель эмбеддингов = cointegrated/LaBSE-en-ru| | |


# Теперь переходим к блоку Reader'а 

### После того, как мы оценили Ретривер, попробуем оценить Ридер. Для этого будем использовать следующие метрики: 

1: Answer Relevancy. Демонстрирует, насколько ответы соответствуют заданным вопросам. Это важный аспект, так как даже правильный ответ с технической точки зрения может быть ненужным, если он не отвечает на конкретный вопрос пользователя. Фактически измеряется как косинусная близость вопроса и ответа

2: Answer Semantic Similarity. Оценивает степень семантической близости между ответом модели и эталонным ответом. Близость ответа и эталонного ответа

3: Используем 2 кастомные метрики (корректность и полнота ответа) и попросим другую LLM оценить наш ответ
Так как у нас есть эталонные вопросы и ответы, нам не составит труда посчитать и оценить данные метрики. 

Но насколько это отражает реальное качество нашей модели? Насколько условия, в которых мы проверяем модель близки к реальным? Насколько эталонные ответы действительно эталонные? 
Дальше попробуем разобраться с этим.


In [ ]:
#Я добавил ячейку, чтобы подгружать данные при повторном воспроизведении ноутбука 
path = '/kaggle/input/questandanswer/evalQA.xlsx'
Q_A_eval = pd.read_excel(path)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
device = 'cuda'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Используем 4-битное квантование
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map=device
)
model.eval()


In [ ]:
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.95,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. Используй только информацию, находяющуюся в контексте и дай развернутый ответ на вопрос. Если ответ не может быть выведен из контекста, не давай ответа.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Теперь вопрос, на который нужно ответить.

Question: {question}""",
    },
]

### Получим от нашей системы ответы на сгенерированные вопрсы

In [ ]:
#Здесь получаю ответы на сгенерированные выше вопросы. 
indexes = [3, 5, 6, 10, 14, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 41, 42, 44, 45, 46, 47, 49, 51, 52, 53, 55, 56, 59, 62, 64, 66, 67, 68, 70, 71, 73, 75, 77, 80, 89, 92, 95, 97, 98, 99, 100, 101, 106, 108, 109, 110, 113, 115, 117, 119, 120, 121, 123, 126, 128, 129, 130, 131, 132, 133, 135, 136, 137, 140, 141, 142, 143, 143, 145, 146]
questions = Q_A_eval.loc[indexes]
dataset = []
for (i,serie) in  questions.iterrows():
    print(i)
    question = serie['question']
    reference = serie['answer']
    user_query = question
    retrieved_docs = vectors_store.similarity_search(query=user_query, k=5)
    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]
    context = "\nExtracted documents:\n"
    context += "".join([f"\nDocument {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
        prompt_in_chat_format, tokenize=False, add_generation_prompt=True
    )
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)
    answer = READER_LLM(final_prompt)[0]["generated_text"]
    dataset.append({"user_input":question,
            "retrieved_contexts":retrieved_docs_text,
            "response":answer,
            "reference":reference
    })

In [ ]:
import json
# Сохранение в JSON файл
with open('dataset.json', 'w', encoding='utf-8') as json_file:
    json.dump(dataset, json_file, ensure_ascii=False, indent=4)
    

In [ ]:
import json
with open('/kaggle/input/questandanswer/dataset.json', 'r', encoding='utf-8') as json_file:
    loaded_dataset = json.load(json_file)
dataset = loaded_dataset    

# Оценка Reader'а

In [ ]:
critique_prompt_completeness = """
Вам будут предоставлены следующие данные: 1) Вопрос, на который нужно ответить. 2)Ответ, который предоставил испытуемый.

Вам нужно определить: насколько ответ соответствует задаваемому вопросу. И поставить оценку ответу. 
1 - Дан ответ не по теме вопроса.  
2 - Ответ косвенно затрагивает тему вопроса.
3 - Дан ответ, который совпадает с темой вопроса, но полностью не отвечает на поставленный вопрос. 
4 - Дан ответ на вопрос, но ответ является поверхностным
5 - Дан исчерпывающий ответ на вопрос. Также ответ ссылается на предоставленные документы
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что ответ не отвечает на вопрос, а 5 означает что дан полный ответ. 

Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 1 до 5
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Ответ на вопрос дан на основе предоставленных документов. Ответ является полным
Оценка: 5

Ниже представлен вопрос и ответ.

Вопрос: {question}
Ответ: {answer}

Output:::
"""


critique_prompt_correctness = """
Вам будут предоставлены следующие данные: 1)предоставленный ответ 2)эталонный ответ. 

Вам нужно определить: насколько данные ответы согласованы. Ваша задача не оценить правильность ответа с фактической точки зрения, а лишь только сравнить предоставленный ответ с эталонным.
1 - Ответы несогласованы. Ответы даны на разные вопросы.
2 - Ответы противоречат друг другу.
3 - Данные ответы содержат общую информацию.
4 - Ответы являются схожими. При этом предоставленный ответ не содержит всей  информации, которая есть в эталонном ответе
5 - Предоставленный ответ содержит всю информацию из правильного ответа. И, возможно, содержит дополнительную информацию, которой нет в эталонном ответе. 
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что два ответа не имеют ничего общего, а 5 означает, что предоставленный ответ содержит всю информацию из эталонного ответа. 


Требования:
1. Строго две строки в ответе
2. Первая строка начинается с "Комментарий: " с кратким обоснованием
3. Вторая строка начинается с "Оценка: " и цифры от 1 до 5
4. Никаких дополнительных пояснений после оценки

Пример правильного ответа:
Комментарий: Предоставленный ответ похож на эталонный. Но не содежит информацию о процентном соотношении, которая есть в эталонном ответе
Оценка: 4

Ниже представлены предоставленный ответ и эталонный ответ.

Предоставленный ответ: {res}
Эталонный ответ: {ref}

Output:::
"""

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import BitsAndBytesConfig

MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"
device = 'cuda'
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Используем 4-битное квантование
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_critique = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map=device
)
model_critique.eval()

tokenizer_critiqe = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Critique_reader = pipeline(
    model=model_critique,
    tokenizer=tokenizer_critiqe,
    task="text-generation",
    do_sample=True,
    temperature=1,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500
)


In [ ]:
import re
Emb = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True} )
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
def cosine_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    # Косинусная близость (не расстояние!)
    cosine_similarity = (np.dot(v1, v2)) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return 1 - cosine_similarity

def extract_score(text):
    # Ищем последнее вхождение паттерна "Оценка: ЦИФРА" в тексте
    matches = re.findall(r'Оценка:\s*(\d+)(?=\D*$)', text, re.IGNORECASE | re.MULTILINE)
    
    if matches:
        # Берем последнюю оценку (на случай, если модель пересмотрела ответ)
        last_score = matches[-1]
        return int(last_score)
    else:
        return None  # или raise ValueError


for i in range(len(dataset)):
    print(i)
    
    quest = dataset[i]["user_input"]
    answer = dataset[i]["response"]
    ref = dataset[i]["reference"]
    
    prompt1 = critique_prompt_completeness.format(question = quest, answer = answer)
    completeness_answer = Critique_reader(prompt1)[0]["generated_text"]
    comp = extract_score(completeness_answer)
    print(comp)
    
    prompt2 = critique_prompt_correctness.format(res = answer, ref = ref)
    correctness_answer = Critique_reader(prompt2)[0]["generated_text"]
    corr = extract_score(correctness_answer)
    print(corr)
    quest_emb = Emb.embed_query(quest)
    answer_emb = Emb.embed_query(answer)
    ref_emb = Emb.embed_query(ref)
    Answer_Relevancy = cosine_distance(quest_emb, answer_emb)
    Answer_Semantic_Similarity = cosine_distance(answer_emb, ref_emb)
    #Если 0, векторы идентичны (чтобы не перепутать косинусную схожесть и косинусное расстояние)
    (dataset[i])['AR'] = Answer_Relevancy
    (dataset[i])['ARS'] = Answer_Semantic_Similarity
    (dataset[i])['completeness'] = comp
    (dataset[i])['correctness'] = corr
    print(Answer_Relevancy, Answer_Semantic_Similarity, comp, corr)

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
#Сохраним файл с оценками Ридера
excel_file_path = '/kaggle/working/Eval_Reader.xlsx'
df.to_excel(excel_file_path, index=False)

In [ ]:
#Скачаем файл с оценками Ридера 
path = '/kaggle/input/questandanswer/Eval_Reader.xlsx'
df = pd.read_excel(path)

In [ ]:
print(df['AR'].mean())
print(df['ARS'].mean())
print(df['completeness'].mean())
print(df['correctness'].mean())

### Отлично. Теперь мы можем оценивать нашу модель. Перебрать гиперпараметры, например k (количество извлекаемых чанков).

### Результаты оценки Ридера

In [ ]:
#Теперь посмотрим, что понаоценивала наша модель. Посмотрим, насколько скоррелированы косинусные расстояние и оценки модели. 
corr = df[['AR', 'ARS', 'completeness', 'correctness']].corr()
corr.style.background_gradient(cmap='coolwarm')
# Очень хорошо, что comp и corr имеют почти нулевую корреляцию. При этом каждая из них имеет отрицательную корреляцию с обоими кос расстояниеями. 
# Получается, comp и corr действительно оценивают ответы нашей системы с разных "точек зрения", а корреляция с кос рас дает надежду на то, что оценки отражают качество нашей модели. 

In [ ]:
#Я вручную отобрал несколько примеров оценки модели: 
print (df.loc[11]["user_input"])
print('----------------')
print (df.loc[11]["response"])
print('----------------')
print (df.loc[11]["reference"])
print (df.loc[11]["completeness"])
print (df.loc[11]["correctness"])

Можем видеть, что модель дала эталонный ответ, но при этом получила не самую высокую оценку. Но в целом, модель очень хорошо отфильтровала бредовые ответы и галюцинации, поставив им единицы (можно посмотреть файл Eval_reader). При этом, хорошоие ответы получили хорошиек оценки. В целом - метрики справляются со своей задачей: Чем лучше модель отвечает нашим запросам - тем выше получает оценки. Таким образом, теперь мы можем варьировать некоторые гиперпараметры и отобрать лучшие комбинации.

# Итоговая оценка Rider'а и лучше гиперпараметры 

Таблица гиперпараметров и метрик

| Модификация | AR | ARS | completeness | correctness
|-------------|-------------|-------------|-----------|-------------
| Модель = saiga_yandexgpt_8b, k-чанков = 3 | 0.3987| 0.3349| 0.3499|  3.1156
| Модель = saiga_yandexgpt_8b, k-чанков = 5 | 0.3841| 0.3511| 0.3636|  3.3122
| Модель = saiga_yandexgpt_8b, k-чанков = 8 | 0.3685| 0.4011| 0.3714|  3.2812 
| Модель = saiga_llama3_8b, k-чанков = 5 | 0.3536| 0.3496| 4.0256|  3.3589
| Модель = GigaChat, k-чанков = 5 | 0| 0| 0| 0


### А итоговую оценку модели мы получим вручную. Для этого я сам написал 10 вопросов и выписал для каждого из вопросов ответы (исключительно на основе документов в базе). Посмотрим, как справится модель с такой задачей 

In [ ]:
#Здесь вопросы, составленные мной вручную для оценки Модели
path = '/kaggle/input/questandanswer/My_questions.xlsx'
my_questions = pd.read_excel(path)

In [ ]:
for i, serie in (my_questions.iterrows()):
    query = serie['Вопрос']
    reference = serie['Ответ ']
    retrieved_docs = vectors_store.similarity_search(query = query, k=5)
    context = "\nExtracted documents:\n"
    context += "".join([f"\nDocument {str(i)}:::\n" + doc.page_content for i, doc in enumerate(retrieved_docs)])
    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
        prompt_in_chat_format, tokenize=False, add_generation_prompt=True
    )
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=query, context=context)
    answer = READER_LLM(final_prompt)[0]["generated_text"]
    print(query)
    print('-------')
    print(reference)
    print('-------')
    print(answer)
    print('\n\n\n')

# Выводы по всей работе: 
Данная RAG отлично работает с той базой знаний, которая у нас есть. Она отлично отвечает на большинство вопросов, и, хочется отметить, практически не выдумывает факты. А отвечает только на основании нашей базы знаний. 
Но если мы начнем задавать модели вопросы по типу "как накачаться?". Она не ответит ничего осмысленного, так как В ДЕЙСТВИТЕЛЬНОСТИ ОТВЕТ НА ВОПРОС НЕ ВСЕГДА БЛИЗОК ПО КОСИНУСНОМУ РАССТОЯНИЮ к вопросу. Так что главный вопрос в том, как мы хотим использовать нашу систему. Если это будет чат-бот для ответов на "общие" вопросы, то в данной работе точно есть, что улучшить.
Но в целом, я доволен тем, что получилось. Так как все статьи написаны в авторском стиле, содержат много сленга и специфических терминов. В статьях много графиков и таблиц и несмотря на все это мы получаем весьма осмысленные ответы. 

### Гипотезы по улучшению: 
- Конечно в идеале для оценки качества работы моделей нужно использовать модели сильнее. Но такой возможности пока нет.
- Можно попробовать делать саммари при загрузке документов в базу знаний. Возможно, саммари поможет находить более релевантные документы
- И, конечно, как только вырастет объем данных, система будет работать гораздо лучше.
- Можно для каждой генерации прикреплять список источников, ссылку на статью, чанк которой был взят и т.п. Для этого просто добавить все эти данные в метадату. 

### 

# Здесь собран финальный пайплайн для использования модели. 

In [ ]:
#Реранкер не помог улучшить качество
'''
from ragatouille import RAGPretrainedModel
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
'''

In [ ]:
from transformers import Pipeline
def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: vectors_store,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Соберём документы с помощью ретривера
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Оставляем только текст

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Финальный промпт
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    print("=> Generating answer...")
    #answer = llm(final_prompt)
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [ ]:
question = "Можно ли есть на ночь? Влияет ли это напрямую на похудение?"

answer, relevant_docs = answer_with_rag(question, READER_LLM, vectors_store, reranker=None)

In [ ]:
print(answer)